In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import torch.nn as nn
import json
import pickle
%cd /content/drive/MyDrive/인공지능/NLP

/content/drive/MyDrive/인공지능/NLP


In [3]:
with open('./MovieReview_SentimentAnalysis-main/top_10000_words.json',encoding='UTF-8') as file:
    data = json.load(file)
    

In [4]:
look_up_table = {word : i for i, word in enumerate(data)}

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
class LSTM(nn.Module):
    def __init__(self,embeded_size):
        super(LSTM,self).__init__()
        self.embeded_size = embeded_size
        self.embedding_layer = nn.Embedding(self.embeded_size,
                                           embedding_dim = 4)
        self.hidden_size = 64
        self.lstm = nn.LSTM(4,self.hidden_size,bidirectional=True,batch_first=True)
        self.linear = nn.Linear(self.hidden_size*2,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x):
        embedding = self.embedding_layer(x)
        lstm_out, (ht,ct) = self.lstm(embedding)
        out = self.linear(lstm_out[:,-1,:])
        return self.sigmoid(out)
        

In [7]:
net = LSTM(len(look_up_table)).to(device)

In [8]:
with open("./MovieReview_SentimentAnalysis-main/trainDocs.pkl","rb") as file:
    data = pickle.load(file)

In [9]:
data_total = data

In [10]:
train_total = []
for data in data_total:
    item_list = []
    for item in data[0]:
        if item in look_up_table:
            item_list.append(look_up_table[item])
    if(len(item_list)>0):
        train_total.append([item_list,data[1]])
train_x = [x[0] for x in train_total]
train_y = [x[1] for x in train_total]
print(len(train_y))

149523


In [15]:
from tqdm.notebook import tqdm

epoch = 20
learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
lossF = torch.nn.BCELoss().to(device)


for i in tqdm(range(0,epoch)):
    b_loss = 0
    for data,label in zip(train_x[:10000],train_y[:10000]):
        data,label = torch.LongTensor([data]).to(device),torch.FloatTensor([[label]]).to(device)
        y_pred = net(data)
        loss = lossF(y_pred,label)
        b_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    accuracy = 0
    for test_data,test_label in zip(train_x[10000:14000],train_y[10000:14000]):
        data = torch.LongTensor([test_data]).to(device)
        y_pred = net(data)
        if y_pred >= 0.5 and test_label == 1:
            accuracy += 1
        if y_pred < 0.5 and test_label == 0:
            accuracy += 1
    print(f'loss : {b_loss/10000} accuracy : {accuracy/len(train_x[10000:14000])}')

loss : 0.5024110078811646 accuracy : 0.7205
loss : 0.4920547306537628 accuracy : 0.722
loss : 0.48239246010780334 accuracy : 0.7305
loss : 0.47310376167297363 accuracy : 0.73175
loss : 0.4642064869403839 accuracy : 0.73975
loss : 0.45519784092903137 accuracy : 0.741
loss : 0.4463166296482086 accuracy : 0.744
loss : 0.43836238980293274 accuracy : 0.74725
loss : 0.43032142519950867 accuracy : 0.754
loss : 0.42264795303344727 accuracy : 0.7605
loss : 0.41602015495300293 accuracy : 0.75925
loss : 0.4087323248386383 accuracy : 0.76275
loss : 0.401888906955719 accuracy : 0.763
loss : 0.39593082666397095 accuracy : 0.76675
loss : 0.38881492614746094 accuracy : 0.76525
loss : 0.38363349437713623 accuracy : 0.766
loss : 0.37771010398864746 accuracy : 0.766
loss : 0.371152400970459 accuracy : 0.77175
loss : 0.36557212471961975 accuracy : 0.7705
loss : 0.35990649461746216 accuracy : 0.77125

